# TabNet Model Test

In this notebook, we'll test a training loop for the TabNet model 


In [1]:
import sys
sys.path.append('../src')

from models.lib.neural import *
from models.lib.data import *
from models.lib.train import *

import helper 
from helper import gene_intersection
from pytorch_tabnet.tab_network import TabNet

import torch.nn as nn 
import torch.optim as optim
import torch
from tqdm import tqdm
from torch.utils.data import Subset
import matplotlib.pyplot as plt 
from models.lib.lightning_train import *

t = helper.INTERIM_DATA_AND_LABEL_FILES_LIST
datafiles, labelfiles = zip(*t.items())
datafiles = [f'../data/interim/{f}' for f in datafiles]
labelfiles = [f'../data/processed/labels/{f}' for f in labelfiles]
refgenes = gene_intersection()

In [2]:
refgenes

['A1BG',
 'A1BG-AS1',
 'A2M',
 'A2M-AS1',
 'A4GALT',
 'AAAS',
 'AACS',
 'AADAT',
 'AAED1',
 'AAGAB',
 'AAK1',
 'AAMDC',
 'AAMP',
 'AANAT',
 'AAR2',
 'AARD',
 'AARS',
 'AARS2',
 'AARSD1',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA10',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD16A',
 'ABHD17A',
 'ABHD17B',
 'ABHD17C',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRACL',
 'ABT1',
 'ABTB1',
 'ABT

In [3]:
train, val, test = generate_dataloaders(
    datafiles=datafiles, 
    labelfiles=labelfiles, 
    class_label='Type', 
    batch_size=4
)
sample = next(iter(train))
sample

/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:513: UserWarning: Extension .csv not recognized,                 interpreting as .csv. To silence this warning, pass in explicit file types.
  warnings.warn(f'Extension {suffix} not recognized, \
/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:513: UserWarning: Extension .csv not recognized,                 interpreting as .csv. To silence this warning, pass in explicit file types.
  warnings.warn(f'Extension {suffix} not recognized, \
/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:513: UserWarning: Extension .csv not recognized,                 interpreting as .csv. To silence this warning, pass in explicit file types.
  warnings.warn(f'Extension {suffix} not recognized, \
/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:513: UserWarning: Extension .csv not rec

(tensor([[2.8467, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.8507, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [1.6067, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([16,  4,  4,  4]))

In [10]:
next(iter(train))[0].shape[1]

19765

In [13]:
model = GeneClassifier(input_dim, output_dim)
model(sample[0])

Model initialized. input_dim = 19765, output_dim = 19. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


tensor([[-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 1.1262, -1.6707, -0.8557, -0.5748, -0.8194, -2.2019,  3.6068, -0.8809,
          2.7857,  1.2517,  2.4645,  2.8015,  0.0755,  0.8277,  0.5843, -1.8812,
          1.1238, -1.1813,  0.2185],
        [-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 0.8386, -2.1398, -3.0723,  1.0672,  0.8691,  3.1868,  1.1563,  0.7659,
          0.0307,  0.4924,  0.3302,  1.6626,  0.8360,  1.0418, -0.3151, -1.0104,
          1.9044, -1.7584, -0.2436]], grad_fn=<MmBackward0>)

In [14]:
for X, y in train:
    print(model(X))

tensor([[-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 1.1262, -1.6707, -0.8557, -0.5748, -0.8194, -2.2019,  3.6068, -0.8809,
          2.7857,  1.2517,  2.4645,  2.8015,  0.0755,  0.8277,  0.5843, -1.8812,
          1.1238, -1.1813,  0.2185],
        [-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 0.8386, -2.1398, -3.0723,  1.0672,  0.8691,  3.1868,  1.1563,  0.7659,
          0.0307,  0.4924,  0.3302,  1.6626,  0.8360,  1.0418, -0.3151, -1.0104,
          1.9044, -1.7584, -0.2436]], grad_fn=<MmBackward0>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

tensor([[-0.3562, -1.1428, -1.0016,  0.5114, -0.8077, -0.8802,  2.9606, -0.9653,
          3.5831,  0.3811,  2.7997,  1.4299,  1.4128, -1.6509, -0.2625, -3.4735,
          0.1468, -2.3657,  1.5537],
        [ 0.2903, -0.5804, -0.2276,  0.0831,  0.0757,  0.0865,  1.4396,  0.4126,
          0.8477,  0.1924,  0.9674,  0.4645,  0.0239,  0.0995, -0.2652, -1.1125,
          0.3964, -1.0295,  0.5315],
        [ 1.0153, -0.8452, -3.4839,  1.7611,  4.4167,  2.5938,  1.1916,  1.0082,
          1.6458,  0.6288,  1.6176,  3.4516,  4.4707,  0.7812,  6.1303, -0.3007,
          5.0950, -0.4546,  4.1602],
        [ 0.2903, -0.5804, -0.2276,  0.0831,  0.0757,  0.0865,  1.4396,  0.4126,
          0.8477,  0.1924,  0.9674,  0.4645,  0.0239,  0.0995, -0.2652, -1.1125,
          0.3964, -1.0295,  0.5315]], grad_fn=<MmBackward0>)
tensor([[ 1.3701e+00, -3.5281e+00, -4.6616e+00,  8.0952e-01,  3.0747e-01,
          1.7324e+00,  3.4454e+00, -4.0639e-01,  1.8236e+00,  1.2802e+00,
          1.8928e+00,  3.1591e+0

tensor([[ 4.7172e-01, -1.6895e+00, -1.8998e+00,  1.1912e+00,  1.1933e+00,
          1.4764e+00,  1.7662e+00,  1.0907e-01,  2.4899e+00,  3.6864e-01,
          2.8090e+00,  2.2477e+00,  3.8412e+00, -1.0378e-01,  3.7934e-01,
         -2.2895e+00,  6.9816e-01, -1.6102e+00,  1.3585e+00],
        [-1.9605e-01, -9.5923e-01, -1.0331e+00,  8.1169e-02, -4.2439e-02,
         -8.1186e-01,  5.8467e-01, -1.2242e+00,  1.5515e+00,  1.0983e+00,
          1.3790e+00,  1.9437e+00,  7.2491e-01,  6.5883e-02,  1.5075e-01,
         -1.5701e+00,  5.9695e-01, -4.3395e-01,  2.8611e-03],
        [ 2.2021e-01, -1.1138e+00, -3.0937e+00,  1.6142e+00,  3.2697e+00,
          2.7354e+00,  1.1502e+00,  9.8554e-01,  1.2605e+00,  5.8303e-01,
          1.5041e+00,  2.0115e+00,  3.8434e+00, -1.9040e-01,  3.8624e+00,
         -1.3856e+00,  3.5473e+00, -9.3684e-01,  3.2201e+00],
        [ 1.0855e+00, -1.6392e+00, -1.3318e+00,  7.1761e-02,  6.4441e-01,
          2.9545e-01,  3.0780e+00,  7.8578e-01,  1.4741e+00,  1.1161e+00,


tensor([[-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 1.1262, -1.6707, -0.8557, -0.5748, -0.8194, -2.2019,  3.6068, -0.8809,
          2.7857,  1.2517,  2.4645,  2.8015,  0.0755,  0.8277,  0.5843, -1.8812,
          1.1238, -1.1813,  0.2185],
        [-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 0.8386, -2.1398, -3.0723,  1.0672,  0.8691,  3.1868,  1.1563,  0.7659,
          0.0307,  0.4924,  0.3302,  1.6626,  0.8360,  1.0418, -0.3151, -1.0104,
          1.9044, -1.7584, -0.2436]], grad_fn=<MmBackward0>)
tensor([[ 0.5010, -1.0437, -1.5717,  0.9378,  1.8747, -0.2664,  2.2228, -0.4981,
          3.8039,  1.1878,  3.2865,  4.2082,  1.2977,  0.7474,  1.7347, -2.9200,
          2.9779, 

tensor([[-0.3433, -0.1743, -1.2748,  0.3542,  0.1061, -0.0909,  0.7551, -0.6208,
          1.0039,  0.3619,  0.0995,  0.8748, -0.4875, -0.6766,  1.3666, -0.9213,
          1.6860, -0.8614,  1.1370],
        [ 0.3317, -2.1533, -3.8927,  0.7024, -0.5503,  0.5381,  2.9809, -1.2235,
          2.2321,  0.9484,  0.9304,  2.4795,  0.8032, -0.7502,  1.9707, -2.0133,
          2.7775, -1.8772,  0.8172],
        [-0.3433, -0.1743, -1.2748,  0.3542,  0.1061, -0.0909,  0.7551, -0.6208,
          1.0039,  0.3619,  0.0995,  0.8748, -0.4875, -0.6766,  1.3666, -0.9213,
          1.6860, -0.8614,  1.1370],
        [ 1.2904, -2.5828, -3.7423,  0.8012,  0.6957,  0.6026,  1.8561, -1.3996,
          2.4041,  1.1756,  2.1806,  4.1264,  4.5194,  0.8161,  2.5663, -0.7936,
          1.9016, -0.1409,  0.0991]], grad_fn=<MmBackward0>)
tensor([[ 0.6589, -0.8940, -0.6892,  0.0699,  0.1424, -1.1702,  2.0562, -0.7036,
          2.5340,  0.7702,  2.1257,  2.5968,  0.5715,  0.5702,  0.7185, -1.5629,
          1.1746, 

...

tensor([[-0.6586,  0.4346, -0.0528,  0.7342,  1.8833,  0.1510, -0.3477,  0.0082,
          1.4463,  0.3768,  1.4618,  1.1349,  1.3521, -0.4045,  1.2759, -1.4583,
          1.1689, -0.5851,  2.2874],
        [ 1.4291, -1.3918, -0.9962,  0.7110,  0.6820,  0.8078,  2.3165,  0.4353,
          2.3645,  0.0118,  2.5018,  2.4892,  2.1803,  1.0604, -0.0995, -1.2678,
          0.6411, -1.7610,  0.8594],
        [-0.5632, -0.8821, -2.4632,  0.4218,  0.2249, -0.5460,  0.9038, -1.6350,
          1.7670,  1.3004,  0.6590,  2.3038,  0.0444, -0.6524,  2.1323, -1.7351,
          2.5795, -0.8210,  1.1105],
        [-0.0501, -0.6641, -1.3659,  0.4862,  0.9267,  0.4642,  0.7905, -0.1082,
          0.9903,  0.6481,  0.8638,  1.3268,  0.8062, -0.0537,  1.2576, -1.1813,
          1.6337, -0.7195,  1.1391]], grad_fn=<MmBackward0>)
tensor([[-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, 

tensor([[ 1.2650, -2.5863, -2.4302, -0.3889,  0.0272, -1.7231,  4.2942, -0.8192,
          2.7753,  1.6900,  3.0114,  2.9445,  3.7409,  0.2040,  3.2432, -1.7480,
          1.7258,  0.1176,  0.7765],
        [ 0.0662, -0.3708, -1.0577,  0.3910,  0.0862,  0.1523,  0.5105, -0.5457,
          0.9791,  0.1949,  0.3819,  1.1578,  0.1964, -0.1156,  0.6605, -0.5622,
          0.9846, -0.7030,  0.5297],
        [-0.1059, -0.7880, -3.5468,  1.3448, -0.0425,  1.6581,  0.5908, -1.3333,
          1.4838,  0.1543, -0.6116,  2.3608, -1.0521, -0.5381,  1.4527, -0.9179,
          3.1824, -2.2567,  1.0383],
        [ 0.0662, -0.3708, -1.0577,  0.3910,  0.0862,  0.1523,  0.5105, -0.5457,
          0.9791,  0.1949,  0.3819,  1.1578,  0.1964, -0.1156,  0.6605, -0.5622,
          0.9846, -0.7030,  0.5297]], grad_fn=<MmBackward0>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

tensor([[-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 1.1262, -1.6707, -0.8557, -0.5748, -0.8194, -2.2019,  3.6068, -0.8809,
          2.7857,  1.2517,  2.4645,  2.8015,  0.0755,  0.8277,  0.5843, -1.8812,
          1.1238, -1.1813,  0.2185],
        [-0.8113, -0.3084, -1.2919,  1.1446,  1.6953,  1.2967,  0.2561,  0.1656,
          1.3897,  0.2519,  1.4429,  0.6376,  2.6771, -1.2848,  1.4546, -1.9468,
          1.1546, -0.8870,  2.3103],
        [ 0.8386, -2.1398, -3.0723,  1.0672,  0.8691,  3.1868,  1.1563,  0.7659,
          0.0307,  0.4924,  0.3302,  1.6626,  0.8360,  1.0418, -0.3151, -1.0104,
          1.9044, -1.7584, -0.2436]], grad_fn=<MmBackward0>)
tensor([[ 2.2021e-01, -1.1138e+00, -3.0937e+00,  1.6142e+00,  3.2697e+00,
          2.7354e+00,  1.1502e+00,  9.8554e-01,  1.2605e+00,  5.8303e-01,
          1.5041e+00,  2.0115e+0

tensor([[ 4.7545e-01, -2.4057e+00, -3.7873e+00,  1.1423e+00,  6.7305e-01,
          1.9384e+00,  9.6851e-01, -9.3123e-01,  1.4950e+00,  1.1502e+00,
          1.2290e+00,  3.2618e+00,  2.0956e+00,  6.2091e-01,  6.2404e-01,
         -1.6431e+00,  2.1069e+00, -1.3898e+00, -1.2850e-01],
        [ 2.9986e-01, -1.1061e+00, -1.6596e+00,  4.5109e-01,  4.0603e-01,
          3.4644e-01,  1.3586e+00, -3.4779e-01,  1.3494e+00,  5.3309e-01,
          1.1814e+00,  1.5257e+00,  1.6765e+00, -1.1652e-01,  1.1772e+00,
         -1.0878e+00,  1.1546e+00, -6.4779e-01,  7.1770e-01],
        [-3.7134e-01,  2.5626e-01, -1.6145e+00,  7.2774e-01,  1.1214e+00,
          2.9438e-01, -3.4819e-01, -8.2876e-01,  9.4423e-01,  3.8533e-01,
         -1.5202e-01,  1.6420e+00, -2.1532e-03, -3.2358e-01,  2.5490e+00,
         -2.7617e-01,  2.5529e+00, -4.3512e-01,  1.6996e+00],
        [ 6.4488e-01, -2.0698e+00, -2.7124e+00,  7.7396e-02, -1.4364e+00,
         -5.3695e-01,  4.0264e+00, -9.0030e-01,  2.3591e+00,  7.1366e-01,


KeyboardInterrupt: 

In [4]:
trainer, model, module = generate_trainer(
    datafiles=datafiles[0:2],
    labelfiles=labelfiles[0:2],
    class_label='Type',
    skip=3,
    normalize=True,
    drop_last=True,
    batch_size=4,
    num_workers=0,
    wandb_name='local-model',
    weighted_metrics=True,
    optim_params={
        'optimizer': torch.optim.SGD,
        'lr': 0.001,
    },
    max_epochs=100,
)

Device is cpu
../data/interim/primary_bhaduri_T.csv exists, continuing...
../data/processed/labels/primary_bhaduri_labels.csv exists, continuing...

../data/interim/allen_cortex_T.csv exists, continuing...
../data/processed/labels/allen_cortex_labels.csv exists, continuing...



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 16604, output_dim = 19. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = True


In [5]:
trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(

  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 1.1 M 
----------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.407     Total estimated model params size (MB)


Done, continuing to training.


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


First, we'll define our train, val and test sets, then generate the associated DataLoaders and try training.

## PyTorch-Lightning compatible TabNet architecture 

In [3]:
data = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', indices=range(0, 20))

train, val, test = generate_single_dataset(datafiles[0], labelfiles[0], 'Type')
train

GeneExpressionData(filename=../data/interim/primary_bhaduri_T.csv, labelname=../data/processed/labels/primary_bhaduri_labels.csv)

In [4]:
train = DataLoader(Subset(train, range(0, 50)), batch_size=4)
val = DataLoader(Subset(val, range(0, 50)), batch_size=4)
test = DataLoader(Subset(test, range(0, 50)), batch_size=4)

In [5]:
model = GeneClassifier(input_dim=19765, output_dim=19)

model.optim_params

Model initialized. input_dim = 19765, output_dim = 19. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


{'optimizer': torch.optim.adam.Adam, 'lr': 0.001, 'weight_decay': 0.01}

In [6]:
from pytorch_lightning import Trainer 
from pytorch_lightning.loggers import WandbLogger
from models.lib.neural import GeneClassifier

wandb_logger = WandbLogger(project='TabNet Cell Type Classifier', name='Tabnet with Metrics')
trainer = Trainer(logger=wandb_logger)

trainer.fit(model, train, val)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 1.3 M 
----------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.241     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:407: UserWarning: The number of training samples (13) is smaller than th

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [2]:
from models.lib.lightning_train import *

In [ ]:
trainer, model, module = generate_trainer(
    
)

In [ ]:
prepare_data('../data')

In [ ]:
t.keys

In [ ]:
t.values()